# SocialAL Model
# Fit model to data - multiple subjects
KLS 11.2.20  
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python Version

In [1]:
import sys
print(sys.version)  

3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
import os
from decimal import *
getcontext().prec = 1000 # increases the precision of data representation 

### Define functions

In [3]:
def update_value(Prob):
    invest = [0,3,6,9]
    retain = [9-x for x in invest] #print ("Retain list is: ", retain)
    shared = [2*x for x in invest] #print ("Shared list is: ", shared)
    EV = [(retain[x] + Prob*shared[x]) for x in range(0,4)]
    return EV

def update_prob(recip, Prob, a_gain, a_loss):
    gain = max(recip - Prob, 0)
    loss = min(recip - Prob, 0)
    Prob = Prob + a_gain * gain + a_loss * loss
    return Prob

def get_action_selection_prob(beta, EV, choice):
    actionProb = Decimal(np.exp(beta*EV[choice-1])/np.sum([np.exp(beta*x) for x in EV]))
    return actionProb

def get_action_selection_probs(beta, EV):
    actionProbs = [get_action_selection_prob(beta, EV, x) for x in range(1,5)]
    return actionProbs

def get_likelihood_action(params, data):
    a_gain = params[0]
    a_loss = params[1]
    beta = params[2]
    decay = params[3]
    
    # initialize variables
    prob = [0.5, 0.5, 0.5]
    ev = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    tletters = ["A", "B", "C"]
    
    totalLLH = 0 
    for trial in range(0, len(data)):

        trustee = data['Stim_Sequence'][trial] # get trustee type
        choice = data['Choice'][trial] # get choice made by participant
        response = data['Trustee_Response'][trial] # get response from trustee
        
        # compute the probability of selecting each option for that trustee
        probs = get_action_selection_probs(beta, ev[trustee])
        
        if choice != 0:
            # use the probability of the selection (choice-probability) to update log likelihood
            cprob = probs[choice-1] #print(cprob, isinstance(cprob, float))
           
            #add to cumulative log likelihood
            totalLLH += -(math.log(cprob))
            

            # update prob and value for trustee 
            if choice != 1:
                prob[trustee] = update_prob(response, prob[trustee], a_gain, a_loss)         
            ev[trustee] = update_value(prob[trustee])
            
        # decay prob and value for other possible trustees 
        options = [0, 1, 2]
        if choice == 2 or choice == 3 or choice == 4:
            options.pop(trustee)
        for x in options:
            prob[x] = prob[x] + decay * (0.5 - prob[x])
            ev[x] =  update_value(prob[x])
    return totalLLH

def model_fit(data):
    
    tries = 10000 #  number of tries to find the best-fit parameter
    lowestLLH = math.inf 
    bestFit = 'NA'
    
    for i in range(tries):
        
        # initialize free parameters with randomly chosen numbers
        a_gain=random.uniform(0, 1)
        a_loss=random.uniform(0, 1)
        beta=random.uniform(0, 1)
        decay=random.uniform(0,1)
        params = [a_gain, a_loss, beta, decay]

        # trying different solvers in the minimize call...
        results = minimize(get_likelihood_action, 
                           params, args =(data), bounds = [(0, 1), (0, 1), (1e-10, 20), (0, 1)], 
                           options = {'maxiter': 10000, 'disp': False})
        if (lowestLLH > results['fun'] and results['success']== True):
            lowestLLH = results['fun']
            bestFit = results
        if bestFit == 'NA':
            line = ['NA', 'NA', 'NA', 'NA', 'NA']
        else:
            line = np.append(bestFit['x'], bestFit['fun'])
      
    return line

### Load and clean data

In [4]:
files = os.listdir('../data/modeling')
files.remove('sub-1027.csv')
files.remove('sub-2008.csv')
files.remove('sub-1031.csv')
files.remove('sub-2014.csv')
files.remove('sub-2015.csv')
files.remove('sub-2016.csv')
files.remove('sub-2029.csv')
files.remove('sub-2032.csv')
files.remove('sub-1040.csv')
younger = [f for f in files if "sub-10" in f]
older = [f for f in files if "sub-20" in f]

In [5]:
def load_and_clean(file):
    path = os.path.join('../data/modeling', file)
    dt = pd.read_csv(path)
    # recode trial type into numbers for model
    def stims(trial_type):
        if trial_type == "Trustworthy":
            return 0
        elif trial_type == "Neutral":
            return 1
        elif trial_type == "Untrustworthy":
            return 2
    dt['Stim_Sequence'] = dt['trial_type'].apply(stims)
    # rename response_key to choice
    def choices(response_key):
        if response_key == 'None':
            return 0 
        else:
            return response_key  
    dt['Choice'] = dt['response_key'].apply(choices)
    dt['Choice'] = pd.to_numeric(dt['Choice'])
    # calculte the trustee response
    def resp(trial_earnings):
        if trial_earnings >= 12:
            return 1
        else:
            return 0
    dt['Trustee_Response'] = dt['trial_earnings'].apply(resp)
    data = dt[['Stim_Sequence','Choice', 'Trustee_Response']]
    return(data)

In [6]:
print(getcontext())
#params = [model_fit(load_and_clean(file)) for file in files]
params = [model_fit(load_and_clean(file)) for file in younger]
#params = [model_fit(load_and_clean(file)) for file in older]

Context(prec=1000, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])


In [7]:
params = pd.DataFrame(params)
#params['id'] = [file[:-4] for file in files]
params['id'] = [file[:-4] for file in younger]
#params['id'] = [file[:-4] for file in older]
# rename columns
params = params.rename({0:'alpha_gain', 1:'alpha_loss', 2:'beta', 3:'decay', 4:'-LLH', 'id':'id'}, axis='columns') 
cols = params.columns.tolist()
cols = cols[-1:] + cols[:-1]
params = params[cols]
params

,id,alpha_gain,alpha_loss,beta,decay,-LLH
0,sub-1009,0.001106,0.001341,20.000000,0.000000,57.040450
1,sub-1021,0.009335,0.025581,20.000000,0.053900,30.456762
2,sub-1035,1.000000,0.535426,0.387557,0.008331,41.896743
3,sub-1034,0.093360,0.190526,2.336965,0.068830,35.488460
4,sub-1020,0.004375,0.054778,20.000000,0.041766,29.820692
5,sub-1008,0.094179,0.987708,0.536531,0.000000,41.593115
6,sub-1036,0.854542,0.145241,0.954389,0.071096,32.237113
7,sub-1022,0.001599,0.015854,20.000000,0.074476,49.649357
8,sub-1023,0.002587,0.012723,20.000000,0.012843,44.353969
9,sub-1037,0.058881,0.054979,20.000000,0.618154,42.287434


In [8]:
# save parameters in text file
#params.to_csv(path_or_buf = '../output/two_alpha_with_decay_model_params.csv', index = False)
params.to_csv(path_or_buf = '../output/two_alpha_with_decay_model_params_younger.csv', index = False)
#params.to_csv(path_or_buf = '../output/two_alpha_with_decay_model_params_older.csv', index = False)